In [2]:
from lxml import html
import requests
from urllib.request import urlopen
import ssl

ssl._create_default_https_context = ssl._create_unverified_context


In [128]:
class DirectoryTree:
    """ Class Providing Constants for Directory Tree.
    """
    title = ""
    parent = ""
    key = ""
    id = ""
    lazy = ""
    folder = ""
    source = ""
    
class DtreeFromHttp():
    """ Class to build server tree
    """
    def __init__(self,path):
        self.__services = []
        self.__listObjects = []
        self.__path = path.strip()
        self.__paperObjects = []
        self.__headers = {
            'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2840.90 Safari/537.36'}

    def fetchForTreeFromHttp(self):
        page = requests.get(self.__path,headers=self.__headers,verify=False)
        tree = html.fromstring(page.content)
        for xtag in tree.xpath('//li/a'):
            file = xtag.text_content().strip()
            if 'Parent Directory' not in file:
                dataFile = DirectoryTree()
                if "qresp.ini" in file:
                    self.openFileToReadConfig("qresp.ini")
                dataFile.title = file
                parent = self.__path.split("/")
                parentName = parent[len(parent) - 2]
                dataFile.key = self.__path + file
                dataFile.id = file
                relPath = str(self.__path + file).split(parentName, 1)[1]
                dataFile.parent = parentName + relPath
                if '/' in file:
                    dataFile.folder = 'true'
                    dataFile.lazy = 'true'
                self.__listObjects.append(dataFile.__dict__)
        return self.__listObjects
    
    def fetchForTreeFromZenodo(self):
        page = requests.get(self.__path,headers=self.__headers,verify=False)
        tree = html.fromstring(page.content)
        for xtag in tree.xpath('//li/a'):
            file = xtag.text_content().strip()
            if 'Parent Directory' not in file:
                dataFile = DirectoryTree()
                if "qresp.ini" in file:
                    self.openFileToReadConfig("qresp.ini")
                dataFile.title = file
                parent = self.__path.split("/")
                parentName = parent[len(parent) - 2]
                dataFile.key = self.__path + file
                dataFile.id = file
                relPath = str(self.__path + file).split(parentName, 1)[1]
                dataFile.parent = parentName + relPath
                if '/' in file:
                    dataFile.folder = 'true'
                    dataFile.lazy = 'true'
                self.__listObjects.append(dataFile.__dict__)
        return self.__listObjects
        
    def openFileToReadConfig(self,configFile):
        try:
            r = urlopen(str(self.__path + configFile))
            for line in r:
                line = str(line)
                if line and "=" in line:
                    linesplit = line.split("=", 1)
                    servicename = str(linesplit[0]).strip()
                    servicepath = str(linesplit[1]).strip()
                    if "http_service_path" in servicename:
                        if servicepath:
                            self.session["fileServerPath"] = servicepath
                            self.session["notebookPath"] = servicepath
                            self.__services.append("http_service_path")
                    elif "globus_service_path" in servicename:
                        if servicepath:
                            self.session["downloadPath"] = servicepath
                            self.__services.append("globus_service_path")
                    elif "isgitservice" in servicename:
                        if servicepath:
                            self.session["gitService"] = servicepath
                            self.__services.append("git_service")
        except Exception as e:
            print("Config file not found",e)

    def checkIsConfigFile(self):
        ssh = ""
        isConfigFile = "N"
        try:
            page = requests.get(self.__path,headers=self.__headers,verify=False)
            tree = html.fromstring(page.content)
            for xtag in tree.xpath('//li/a'):
                file = xtag.text_content().strip()
                if 'Parent Directory' not in file:
                    if "qresp.ini" in file:
                        isConfigFile = "Y"
                        break
        except:
            print("Could not connect to remote server")
        return isConfigFile

    def fetchServices(self):
        return self.__services

    

In [129]:
dtree = DtreeFromHttp('https://notebook.rcc.uchicago.edu/files/')
opt = dtree.fetchForTreeFromHttp()

l> b'[SERVICE]\n'
l> b'ishttpservice = Y\n'
l> b'isglobusservice = Y\n'
l> b'isgitservice = Y\n'
l> b'\n'
l> b'[SERVICE.PATH]\n'
l> b'http_service_path=https://notebook.rcc.uchicago.edu/files\n'
l> b'globus_service_path=https://app.globus.org/file-manager?origin_id=72277ed4-1ad3-11e7-bbe1-22000b9a448b&origin_path=\n'


C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:857: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


In [10]:
headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2840.90 Safari/537.36'}
page = requests.get('https://zenodo.org/record/3188334#.XPFinohKhPY',headers=headers,verify=False)
tree = html.fromstring(page.content)
for xtag in tree.xpath('//table[@class="table table-striped"]//td/a'):
    file = xtag.text_content().strip()
    print(file)

C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:857: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


GTDB_bac-arc_ssu_r86.fa.gz
RefSeq-RDP16S_v3_May2018.fa.gz
Version2AffectedSeqs.txt


DAT

In [66]:
from urllib import urlopen
headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2840.90 Safari/537.36'}
page = requests.get('https://datbase.org/dat://6e2a47fba1d1db85836c6920f1055cc3b4c3a6b52050848cd967febfdf3915d4',headers=headers,verify=False)
url = 'https://datbase.org/dat://6e2a47fba1d1db85836c6920f1055cc3b4c3a6b52050848cd967febfdf3915d4'
tree = etree.HTML(urlopen(url).read())
tree = html.fromstring(page.content)
for xtag in tree.xpath('//div[@id="app"]//div//div[@class="_41b819fa"]//main[@class="site-main"]//div'):
    print("0",xtag)
    file = xtag.text_content().strip()
    print(file)

ImportError: cannot import name 'urlopen'

In [ ]:
from urllib import urlopen
headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2840.90 Safari/537.36'}
page = requests.get('https://datbase.org/dat://6e2a47fba1d1db85836c6920f1055cc3b4c3a6b52050848cd967febfdf3915d4',headers=headers,verify=False)
url = 'https://datbase.org/dat://6e2a47fba1d1db85836c6920f1055cc3b4c3a6b52050848cd967febfdf3915d4'
tree = etree.HTML(urlopen(url).read())
tree = html.fromstring(page.content)
for xtag in tree.xpath('//div[@id="app"]//div//div[@class="_41b819fa"]//main[@class="site-main"]//div'):
    print("0",xtag)
    file = xtag.text_content().strip()
    print(file)

In [13]:
headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2840.90 Safari/537.36'}
page = requests.get('https://www.zenodo.org/record/3332726#.XSdxFuhKhPY',headers=headers,verify=False)
tree = html.fromstring(page.content)



for xtag in tree.xpath('//a'):
    print(xtag)
    file = xtag.text_content().strip()
    print(file)

C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:851: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


<Element a at 0x26d6bd3fd68>

<Element a at 0x26d6bd7c4a8>
Upload
<Element a at 0x26d6bd984a8>
Communities
<Element a at 0x26d6bd982c8>
Log in
<Element a at 0x26d6bd98638>
Sign up
<Element a at 0x26d6bd98728>
Preview
<Element a at 0x26d6bd98598>
Files
<Element a at 0x26d6bd987c8>
Colex_Europe_Suppl.zip
<Element a at 0x26d6bd98868>
Download
<Element a at 0x26d6bd988b8>
Beta
<Element a at 0x26d6bd98908>
Citations
<Element a at 0x26d6bd98a48>
See more details...
<Element a at 0x26d6bd98ae8>
More info on how stats are collected.
<Element a at 0x26d6bd98a98>

<Element a at 0x26d6bd98b88>
Creative Commons Attribution 4.0 International
<Element a at 0x26d6bd98b38>
Version 1
<Element a at 0x26d6bd98bd8>
10.5281/zenodo.3332725
<Element a at 0x26d6bd98818>
Read more
<Element a at 0x26d6bd952c8>

<Element a at 0x26d6bd95c78>

<Element a at 0x26d6bd95818>

<Element a at 0x26d6bd954f8>

<Element a at 0x26d6bd95c28>

<Element a at 0x26d6bd95d18>

<Element a at 0x26d6bd95d68>
BibTeX
<Element a at 0x2

In [ ]:
for iframe in iframexx:
    response = urllib2.urlopen(iframe.attrs['src'])
    iframe_soup = BeautifulSoup(response)

In [20]:
from lxml.etree import fromstring
import urllib
headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2840.90 Safari/537.36'}
page = requests.get('https://www.zenodo.org/record/3332726#.XSdxFuhKhPY',headers=headers,verify=False)
tree = html.fromstring(page.content)
src_url = tree.cssselect("iframe").attrib['src']
data = urllib.urlopen(src_url) # open iframe src url
tree = fromstring(data.read()) # parsing iframe tree

C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:851: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


ImportError: cssselect does not seem to be installed. See http://packages.python.org/cssselect/

In [25]:
import lxml.html as lh
import requests


rq= requests.get('https://www.zenodo.org/record/3332726#.XSdxFuhKhPY')

content = rq.content

doc = lh.fromstring(content)

t = doc.xpath('//[@id="preview-iframe"]/iframe')
url_data = t.attrib.get('src')
data = requests.get(url_data).content

XPathEvalError: Invalid expression

In [ ]:
headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2840.90 Safari/537.36'}
page = requests.get('https://www.zenodo.org/record/3332726#.XSdxFuhKhPY',headers=headers,verify=False)
tree = html.fromstring(page.content)



for xtag in tree.xpath('//a'):
    print(xtag)
    file = xtag.text_content().strip()
    print(file)

In [38]:
import urllib.request 
import urllib.parse 
from lxml.etree import fromstring
data = urllib.request.urlopen('https://www.zenodo.org/record/3332726#.XSdxFuhKhPY') # fetching url with iframe
tree = html.fromstring(data.read()) # parsing tree to get src attribute
print(tree)
src_url = tree.cssselect("iframe")[0].attrib['src']
print(src_url)
data = urllib.request.urlopen('https://www.zenodo.org/record/3332726#.XSdxFuhKhPY'+src_url) # open iframe src url
tree = html.fromstring(data.read()) # parsing iframe tree
print(tree.text_content().strip())

<Element html at 0x26d6be1f778>
/record/3332726/preview/Colex_Europe_Suppl.zip
Supplementary materials colexifications Europe | Zenodo
    
    
    
    
    




  
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
  
  
    


  
    
      
        
          Toggle navigation
          
          
          
        
          
      
      

  
    
      
      
        
      
    
  

        
            Upload
            Communities
        

   Log in
   Sign up

      
    
  




  
    
      
        July 11, 2019
        
          
Dataset
          

Open Access

        
      
      Supplementary materials colexifications Europe
      
Anonymous
      Supplementary materials accompanying submission on colexification patterns in Europe


  
    
      
        Preview
        
        
      
    
    
  
    
  
  
    
      Files
      
      
    
     (81.1 MB)
  
  
  
    
      
        Name
        Size
      
    
    
      

complete-sample.png
european-sample.png
heatmap-colex.png
heatmap-family-relations.png
heatmap-nbnet.png
heatmap-phylo.png
model-comparison.png
pcoa-gen.png
pcoa-month-moon.png
pcoa-nbnet-color-paint.png
pcoa-nbnet.png
pcoa-sample.png
sample-dendrogramm.png
sample-network.png
svm-MONTH.MOON.png
svm-color.paint.png
colex-changes
regr


In [90]:
class DirectoryTree:
    """ Class Providing Constants for Directory Tree.
    """
    title = ""
    parent = ""
    key = ""
    id = ""
    lazy = ""
    folder = ""
    source = ""

In [ ]:
page = requests.get(self.__path,headers=self.__headers,verify=False)
tree = html.fromstring(page.content)
for xtag in tree.xpath('//li/a'):
    file = xtag.text_content().strip()
    if 'Parent Directory' not in file:
        dataFile = DirectoryTree()
        dataFile.title = file
        parent = self.__path.split("/")
        parentName = parent[len(parent) - 2]
        dataFile.key = self.__path + file
        dataFile.id = file
        relPath = str(self.__path + file).split(parentName, 1)[1]
        dataFile.parent = parentName + relPath
        if '/' in file:
            dataFile.folder = 'true'
            dataFile.lazy = 'true'
        self.__listObjects.append(dataFile.__dict__)

In [122]:
def extractFromZenodo(attrid,url):
    listObjects = []
    page = requests.get(url) # open iframe src url
    tree = html.fromstring(page.content)
    #tree = html.fromstring(data.read()) # parsing iframe tree
    tag = '//ul[@id="'+ attrid + '"]/li/span[1]'
    for xtag in tree.xpath(tag):
        file = xtag.text_content().strip()
        print(file)
        dataFile = DirectoryTree()
        dataFile.title = file
        dataFile.key = url + '#' + attrid
        dataFile.id = file
        dataFile.parent = attrid
    listObjects.append(dataFile.__dict__)
    tag = '//ul[@id="'+ attrid + '"]/li/a[1]'
    for xtag in tree.xpath(tag):
        file = xtag.text_content().strip()
        href = xtag.xpath("@href")[0].split("#")[1]
        print("links ",file, href)
        dataFile = DirectoryTree()
        dataFile.title = file
        dataFile.key = url + '#' + attrid
        dataFile.id = href
        dataFile.parent = attrid
        dataFile.folder = 'true'
        dataFile.lazy = 'true'
    url =     
    page = requests.get('https://www.zenodo.org/record/3333915') # open iframe src url
    tree = html.fromstring(page.content)
    tag = '//ul[@id="'+ attrid + '"]/li/a[1]'
    for xtag in tree.xpath(tag):
        file = xtag.text_content().strip()
        href = xtag.xpath("@href")[0].split("#")[1]
        print("links ",file, href)
        dataFile = DirectoryTree()
        dataFile.title = file
        dataFile.key = url + '#' + attrid
        dataFile.id = href
        dataFile.parent = attrid
        dataFile.folder = 'true'
        dataFile.lazy = 'true'
    listObjects.append(dataFile.__dict__)
    return listObjects

In [123]:
print(extractFromZenodo('tree_item36','https://www.zenodo.org/record/3332726/preview/Colex_Europe_Suppl.zip'))

complete-sample.png
european-sample.png
heatmap-colex.png
heatmap-family-relations.png
heatmap-nbnet.png
heatmap-phylo.png
model-comparison.png
pcoa-gen.png
pcoa-month-moon.png
pcoa-nbnet-color-paint.png
pcoa-nbnet.png
pcoa-sample.png
sample-dendrogramm.png
sample-network.png
svm-MONTH.MOON.png
svm-color.paint.png
links  colex-changes tree_item37
links  regr tree_item447
[{'title': 'svm-color.paint.png', 'key': 'https://www.zenodo.org/record/3332726/preview/Colex_Europe_Suppl.zip#tree_item36', 'id': 'svm-color.paint.png', 'parent': 'tree_item36'}, {'title': 'regr', 'key': 'https://www.zenodo.org/record/3332726/preview/Colex_Europe_Suppl.zip#tree_item36', 'id': 'tree_item447', 'parent': 'tree_item36', 'folder': 'true', 'lazy': 'true'}]


In [134]:
page = requests.get('https://www.zenodo.org/record/3333915') # open iframe src url
tree = html.fromstring(page.content)
tag = '//div[@id="files"]/div[2]/table/tbody/tr/td/a'
for xtag in tree.xpath(tag):
    print(xtag)
    file = xtag.text_content().strip()
    print("links ",file)

<Element a at 0x26d6bd95f48>
links  156.jpg
